In [1]:
# import libraries

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
color = sns.color_palette()
import lightgbm as lgb 
from sklearn.metrics import mean_absolute_error

In [2]:
# import data

prop = pd.read_csv('C:\\Users\\Sonja\\Zillow\\properties_2016.csv',\
                  low_memory=False)
train = pd.read_csv('C:\\Users\\Sonja\\Zillow\\train_2016.csv',\
                  low_memory=False)
sample = pd.read_csv('C:\\Users\\Sonja\\Zillow\\sample_submission.csv',\
                  low_memory=False)


In [3]:
# merge dataframes

df_train = train.merge(prop, how='left', on='parcelid')

In [4]:
# fill missing values with zero
df_train_new = df_train.fillna(0)

In [5]:
#Encode columns containing object data types

from sklearn import preprocessing 

le = preprocessing.LabelEncoder()

df_train_new['hashottuborspa'] = le.fit_transform(df_train_new['hashottuborspa'])
df_train_new['fireplaceflag'] = le.fit_transform(df_train_new['fireplaceflag'])

df_train_new['propertyzoningdesc'] = df_train_new['propertyzoningdesc'].factorize()[0]
df_train_new['propertyzoningdesc'] = le.fit_transform(df_train_new['propertyzoningdesc'])

df_train_new['propertycountylandusecode'] = df_train_new['propertycountylandusecode'].factorize()[0]
df_train_new['propertycountylandusecode'] = le.fit_transform(df_train_new['propertycountylandusecode'])

df_train_new['taxdelinquencyflag'] = df_train_new['taxdelinquencyflag'].factorize()[0]
df_train_new['taxdelinquencyflag'] = le.fit_transform(df_train_new['taxdelinquencyflag'])

In [6]:
df_train_new.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,0.0,0.0,2.0,3.0,0.0,4.0,...,0.0,0,122754.0,360170.0,2015.0,237416.0,6735.88,0,0.0,6.037107e+13
1,14366692,-0.1684,2016-01-01,0.0,0.0,0.0,3.5,4.0,0.0,0.0,...,0.0,0,346458.0,585529.0,2015.0,239071.0,10153.02,0,0.0,0.000000e+00
2,12098116,-0.0040,2016-01-01,1.0,0.0,0.0,3.0,2.0,0.0,4.0,...,0.0,0,61994.0,119906.0,2015.0,57912.0,11484.48,0,0.0,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,0.0,0.0,2.0,2.0,0.0,4.0,...,0.0,0,171518.0,244880.0,2015.0,73362.0,3048.74,0,0.0,6.037296e+13
4,14432541,-0.0050,2016-01-02,0.0,0.0,0.0,2.5,4.0,0.0,0.0,...,2.0,0,169574.0,434551.0,2015.0,264977.0,5488.96,0,0.0,6.059042e+13


**Feature engineering on training set**

In [7]:
# create new features distinguishing between zero and non-zero values in df_train_new
#if df[column].isnull().sum(axis=0)>= 200000:

def null_values(column):
    if column == 0:
        return 0
    elif column != 0:
        return 1    
                
df_train_new['feat_storytypeid'] = df_train_new['storytypeid'].apply(null_values)
df_train_new['feat_basementsqft'] = df_train_new['basementsqft'].apply(null_values)
df_train_new['feat_yardbuildingsqft26'] = df_train_new['yardbuildingsqft26'].apply(null_values)
df_train_new['feat_fireplaceflag'] = df_train_new['fireplaceflag'].apply(null_values)
df_train_new['feat_architecturalstyletypeid'] = df_train_new['architecturalstyletypeid'].apply(null_values)
df_train_new['feat_typeconstructiontypeid'] = df_train_new['typeconstructiontypeid'].apply(null_values)
df_train_new['feat_finishedsquarefeet13'] = df_train_new['finishedsquarefeet13'].apply(null_values)
df_train_new['feat_buildingclasstypeid'] = df_train_new['buildingclasstypeid'].apply(null_values)
df_train_new['feat_decktypeid'] = df_train_new['decktypeid'].apply(null_values)
df_train_new['feat_finishedsquarefeet6'] = df_train_new['finishedsquarefeet6'].apply(null_values)
df_train_new['feat_poolsizesum'] = df_train_new['poolsizesum'].apply(null_values)
df_train_new['feat_pooltypeid2'] = df_train_new['pooltypeid2'].apply(null_values)
df_train_new['feat_pooltypeid10'] = df_train_new['pooltypeid10'].apply(null_values)
df_train_new['feat_taxdelinquencyflag'] = df_train_new['taxdelinquencyflag'].apply(null_values)
df_train_new['feat_taxdelinquencyyear'] = df_train_new['taxdelinquencyyear'].apply(null_values)
df_train_new['feat_hashottuborspa'] = df_train_new['hashottuborspa'].apply(null_values)
df_train_new['feat_yardbuildingsqft17'] = df_train_new['yardbuildingsqft17'].apply(null_values)
df_train_new['feat_finishedsquarefeet15'] = df_train_new['finishedsquarefeet15'].apply(null_values)
df_train_new['feat_finishedsquarefeet50'] = df_train_new['finishedsquarefeet50'].apply(null_values)
df_train_new['feat_finishedfloor1squarefeet'] = df_train_new['finishedfloor1squarefeet'].apply(null_values)
df_train_new['feat_threequarterbathnbr'] = df_train_new['threequarterbathnbr'].apply(null_values)
df_train_new['feat_fireplacecnt'] = df_train_new['fireplacecnt'].apply(null_values)
df_train_new['feat_pooltypeid7'] = df_train_new['pooltypeid7'].apply(null_values)
df_train_new['feat_poolcnt'] = df_train_new['poolcnt'].apply(null_values)
df_train_new['feat_numberofstories'] = df_train_new['numberofstories'].apply(null_values)
df_train_new['feat_airconditioningtypeid'] = df_train_new['airconditioningtypeid'].apply(null_values)
df_train_new['feat_garagetotalsqft'] = df_train_new['garagetotalsqft'].apply(null_values)
df_train_new['feat_garagecarcnt'] = df_train_new['garagecarcnt'].apply(null_values)
df_train_new['feat_regionidneighborhood'] = df_train_new['regionidneighborhood'].apply(null_values)
df_train_new['feat_heatingorsystemtypeid'] = df_train_new['heatingorsystemtypeid'].apply(null_values)
df_train_new['feat_buildingqualitytypeid'] = df_train_new['buildingqualitytypeid'].apply(null_values)
df_train_new['feat_unitcnt'] = df_train_new['unitcnt'].apply(null_values)
df_train_new['feat_propertyzoningdesc'] = df_train_new['propertyzoningdesc'].apply(null_values)
df_train_new['feat_lotsizesquarefeet'] = df_train_new['lotsizesquarefeet'].apply(null_values)
df_train_new['feat_finishedsquarefeet12'] = df_train_new['finishedsquarefeet12'].apply(null_values)
df_train_new['feat_calculatedbathnbr'] = df_train_new['calculatedbathnbr'].apply(null_values)
df_train_new['feat_structuretaxvaluedollarcnt'] = df_train_new['structuretaxvaluedollarcnt'].apply(null_values)
df_train_new['feat_taxvaluedollarcnt'] = df_train_new['taxvaluedollarcnt'].apply(null_values)
df_train_new['feat_taxamount'] = df_train_new['taxamount'].apply(null_values)
df_train_new['feat_regionidzip'] = df_train_new['regionidzip'].apply(null_values)
df_train_new['feat_propertycountylandusecode'] = df_train_new['propertycountylandusecode'].apply(null_values)
df_train_new['feat_roomcnt'] = df_train_new['roomcnt'].apply(null_values)
df_train_new['feat_bathroomcnt'] = df_train_new['bathroomcnt'].apply(null_values)
df_train_new['feat_bedroomcnt'] = df_train_new['bedroomcnt'].apply(null_values)
df_train_new['feat_assessmentyear'] = df_train_new['assessmentyear'].apply(null_values)
df_train_new['feat_regionidcounty'] = df_train_new['regionidcounty'].apply(null_values)
df_train_new['feat_fips'] = df_train_new['fips'].apply(null_values)
df_train_new['feat_latitude'] = df_train_new['latitude'].apply(null_values)
df_train_new['feat_longitude'] = df_train_new['longitude'].apply(null_values)
df_train_new['feat_propertylandusetypeid'] = df_train_new['propertylandusetypeid'].apply(null_values)
df_train_new['feat_rawcensustractandblock'] = df_train_new['rawcensustractandblock'].apply(null_values)

In [8]:
# define predictors (X) and target (y)

#y = pd.DataFrame(data['logerror'])
y=df_train_new['logerror'].ravel()

In [9]:
X = df_train_new.drop('logerror', axis=1, inplace=True)
X=df_train_new.drop('transactiondate', axis=1, inplace=True)
X = df_train_new
X.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,feat_roomcnt,feat_bathroomcnt,feat_bedroomcnt,feat_assessmentyear,feat_regionidcounty,feat_fips,feat_latitude,feat_longitude,feat_propertylandusetypeid,feat_rawcensustractandblock
0,11016594,1.0,0.0,0.0,2.0,3.0,0.0,4.0,2.0,0.0,...,0,1,1,1,1,1,1,1,1,1
1,14366692,0.0,0.0,0.0,3.5,4.0,0.0,0.0,3.5,0.0,...,0,1,1,1,1,1,1,1,1,1
2,12098116,1.0,0.0,0.0,3.0,2.0,0.0,4.0,3.0,0.0,...,0,1,1,1,1,1,1,1,1,1
3,12643413,1.0,0.0,0.0,2.0,2.0,0.0,4.0,2.0,0.0,...,0,1,1,1,1,1,1,1,1,1
4,14432541,0.0,0.0,0.0,2.5,4.0,0.0,0.0,2.5,0.0,...,1,1,1,1,1,1,1,1,1,1


In [10]:
# split dataset

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [11]:
#Encode columns containing object data types

from sklearn import preprocessing 

le = preprocessing.LabelEncoder()

prop['hashottuborspa'] = le.fit_transform(prop['hashottuborspa'])
prop['fireplaceflag'] = le.fit_transform(prop['fireplaceflag'])

prop['propertyzoningdesc'] = prop['propertyzoningdesc'].factorize()[0]
prop['propertyzoningdesc'] = le.fit_transform(prop['propertyzoningdesc'])

prop['propertycountylandusecode'] = prop['propertycountylandusecode'].factorize()[0]
prop['propertycountylandusecode'] = le.fit_transform(prop['propertycountylandusecode'])

prop['taxdelinquencyflag'] = prop['taxdelinquencyflag'].factorize()[0]
prop['taxdelinquencyflag'] = le.fit_transform(prop['taxdelinquencyflag'])

C:\Users\Sonja\Anaconda2\lib\site-packages\numpy\lib\arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [12]:
# fill missing values with zero

prop = prop.fillna(0)
prop.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,9.0,2015.0,9.0,0.00,0,0.0,0.0
1,10759547,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6476,0.0,27516.0,2015.0,27516.0,0.00,0,0.0,0.0
2,10843547,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6476,650756.0,1413387.0,2015.0,762631.0,20800.37,0,0.0,0.0
3,10859147,0.0,0.0,0.0,0.0,0.0,3.0,7.0,0.0,0.0,...,1.0,6476,571346.0,1156834.0,2015.0,585488.0,14557.57,0,0.0,0.0
4,10879947,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,6476,193796.0,433491.0,2015.0,239695.0,5725.17,0,0.0,0.0


**Replicate feature engineering on prediction file**

In [13]:
# create new features distinguishing between zero and non-zero values in prop

prop['feat_storytypeid'] = prop['storytypeid'].apply(null_values)
prop['feat_basementsqft'] = prop['basementsqft'].apply(null_values)
prop['feat_yardbuildingsqft26'] = prop['yardbuildingsqft26'].apply(null_values)
prop['feat_fireplaceflag'] = prop['fireplaceflag'].apply(null_values)
prop['feat_architecturalstyletypeid'] = prop['architecturalstyletypeid'].apply(null_values)
prop['feat_typeconstructiontypeid'] = prop['typeconstructiontypeid'].apply(null_values)
prop['feat_finishedsquarefeet13'] = prop['finishedsquarefeet13'].apply(null_values)
prop['feat_buildingclasstypeid'] = prop['buildingclasstypeid'].apply(null_values)
prop['feat_decktypeid'] = prop['decktypeid'].apply(null_values)
prop['feat_finishedsquarefeet6'] = prop['finishedsquarefeet6'].apply(null_values)
prop['feat_poolsizesum'] = prop['poolsizesum'].apply(null_values)
prop['feat_pooltypeid2'] = prop['pooltypeid2'].apply(null_values)
prop['feat_pooltypeid10'] = prop['pooltypeid10'].apply(null_values)
prop['feat_taxdelinquencyflag'] = prop['taxdelinquencyflag'].apply(null_values)
prop['feat_taxdelinquencyyear'] = prop['taxdelinquencyyear'].apply(null_values)
prop['feat_hashottuborspa'] = prop['hashottuborspa'].apply(null_values)
prop['feat_yardbuildingsqft17'] = prop['yardbuildingsqft17'].apply(null_values)
prop['feat_finishedsquarefeet15'] = prop['finishedsquarefeet15'].apply(null_values)
prop['feat_finishedsquarefeet50'] = prop['finishedsquarefeet50'].apply(null_values)
prop['feat_finishedfloor1squarefeet'] = prop['finishedfloor1squarefeet'].apply(null_values)
prop['feat_threequarterbathnbr'] = prop['threequarterbathnbr'].apply(null_values)
prop['feat_fireplacecnt'] = prop['fireplacecnt'].apply(null_values)
prop['feat_pooltypeid7'] = prop['pooltypeid7'].apply(null_values)
prop['feat_poolcnt'] = prop['poolcnt'].apply(null_values)
prop['feat_numberofstories'] = prop['numberofstories'].apply(null_values)
prop['feat_airconditioningtypeid'] = prop['airconditioningtypeid'].apply(null_values)
prop['feat_garagetotalsqft'] = prop['garagetotalsqft'].apply(null_values)
prop['feat_garagecarcnt'] = prop['garagecarcnt'].apply(null_values)
prop['feat_regionidneighborhood'] = prop['regionidneighborhood'].apply(null_values)
prop['feat_heatingorsystemtypeid'] = prop['heatingorsystemtypeid'].apply(null_values)
prop['feat_buildingqualitytypeid'] = prop['buildingqualitytypeid'].apply(null_values)
prop['feat_unitcnt'] = prop['unitcnt'].apply(null_values)
prop['feat_propertyzoningdesc'] = prop['propertyzoningdesc'].apply(null_values)
prop['feat_lotsizesquarefeet'] = prop['lotsizesquarefeet'].apply(null_values)
prop['feat_finishedsquarefeet12'] = prop['finishedsquarefeet12'].apply(null_values)
prop['feat_calculatedbathnbr'] = prop['calculatedbathnbr'].apply(null_values)
prop['feat_structuretaxvaluedollarcnt'] = prop['structuretaxvaluedollarcnt'].apply(null_values)
prop['feat_taxvaluedollarcnt'] = prop['taxvaluedollarcnt'].apply(null_values)
prop['feat_taxamount'] = prop['taxamount'].apply(null_values)
prop['feat_regionidzip'] = prop['regionidzip'].apply(null_values)
prop['feat_propertycountylandusecode'] = prop['propertycountylandusecode'].apply(null_values)
prop['feat_roomcnt'] = prop['roomcnt'].apply(null_values)
prop['feat_bathroomcnt'] = prop['bathroomcnt'].apply(null_values)
prop['feat_bedroomcnt'] = prop['bedroomcnt'].apply(null_values)
prop['feat_assessmentyear'] = prop['assessmentyear'].apply(null_values)
prop['feat_regionidcounty'] = prop['regionidcounty'].apply(null_values)
prop['feat_fips'] = prop['fips'].apply(null_values)
prop['feat_latitude'] = prop['latitude'].apply(null_values)
prop['feat_longitude'] = prop['longitude'].apply(null_values)
prop['feat_propertylandusetypeid'] = prop['propertylandusetypeid'].apply(null_values)
prop['feat_rawcensustractandblock'] = prop['rawcensustractandblock'].apply(null_values)

In [14]:
#predict on columns contained in X

prop=prop[X.columns]


**XGBoost model**

In [15]:
# Run XGBoost model on train & test sets

trn_x = X_train
trn_y = y_train
val_x = X_test
val_y = y_test

train_data = lgb.Dataset(trn_x, trn_y) # trn_x put the train dataset and its target
test_data = lgb.Dataset(val_x,val_y, reference=train_data) #val is the test set that you use

In [17]:
# setting up parameters - more details at https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters.md

params = {
    'boosting_type':'gbdt',
    'objective':'regression',
    'metric':'mae',
    'sub_feature':0.6,
    'num_leaves':30,
    'min_data':250,
    'min_hessian':1,
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'num_iterations':80, 
    'is_unbalanced':False,
    'early_stopping_round':5 # stops if no improvement after 5 iterations
}  

num_boost_round = 250 # number of iterations

lgb.cv(params, train_data, num_boost_round, nfold=5) # crossvalidates with 5 folds for selection
bst = lgb.train(params, train_data, num_boost_round, valid_sets=[test_data] )
print(bst.best_iteration)

WindowsError: exception: access violation reading 0x0000000000000040

**Keras model**

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
from keras import optimizers
from keras.callbacks import EarlyStopping


ImportError: No module named keras.models

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [ ]:
# convert imported data into numpy arrays

y_train = np.array(y_train)
X_train = np.array(X_train)
y_test = np.array(y_test)
X_test = np.array(X_test)

prop = np.array(prop)

n_cols = X_train.shape[1]

In [ ]:
# baseline model

model = Sequential()
model.add(Dense(32,input_shape=(n_cols,), kernel_initializer='uniform', activation='softmax'))
model.add(Dropout(0.5))
#model.add(Dense(100, activation='softmax'))
#model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))

#opt = optimizers.sgd(lr=0.01)
early_stopping_monitor = EarlyStopping(patience=2)

model.compile(loss='binary_crossentropy',\
              optimizer='adam',class_mode='binary', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.3,\
         callbacks=[early_stopping_monitor])

In [ ]:
pred_keras = model.predict(prop)

**Predictions on XGBoost model**

In [ ]:
#Predict on prop file
pred_xgboost = bst.predict(prop, num_iteration=bst.best_iteration)


In [ ]:
print("Start writing to file ...")
for c in sample.columns[sample.columns != 'ParcelId']:
    sample[c] = pred_xgboost


In [ ]:
sample.to_csv('submission.csv', index=False, float_format='%.4f')

**Stack XGBoost and Keras model**